# Use Publication PDFs to extract additional data for classification and filtering.
The for publications in the UK Catalysis Hub app db contains a titles, IDs and DOIs. We can use PDF miner to look for keywords in the file. Wa can also use Chem Data Extractor to get lists of terms which can also be used to classify the publications.

The steps of the process are: 
 1. get a Title, DOI, URL and pdf file name for each publication
 2. get a list of terms (keywords) to search for
 3. open a pdf file
 4. search for the terms using PDFMiner
 5. store terms found with a count of occurences per publication, assuming that repeated mentions imply relevance of term.
 6. search for terms using chem data extractor
 7. store chem data extractor terms with a counter of occurences, again assuming that repeated mentions indicate relevance
 

In [1]:
# managing files and file paths
from pathlib import Path

# import pdfminer functions
from pdfminer import high_level as pdfmnr_hl

# add aprogress bar
from tqdm import tqdm_notebook 

# custom libraries
# import custom functions (common to various notebooks)
import processing_functions as pr_fns
# library containign functions that read and write to csv files
import lib.handle_csv as csvh

## 1. Get list of publications from DB
Open publications DB and get list of publications including Title, ID and DOI.
The list will be matched against list of publications. If the data file contains title in top pages, then it must be the article, proceed to analise in step 2


In [2]:


# Set the name of currend app DB
ukchapp_db = "db_files/app_db20211216.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()

# Get publication data from the ukch app
db_pubs = pr_fns.get_pub_data(ukchapp_db)

## 2. Define list of search terms
The initial list consists of experimental techniques commonly mentioned in UKCH papers.

In [3]:
#List of terms
list_of_techniques = ['STEM', 'Scanning transmission electron microscopy',
                      'TEM ','Transmission Electron Microscopy',
                      'UV-VIS','Ultraviolet–visible spectroscopy','ultraviolet–visible spectrophotometry',
                      'TGA','Thermogravimetric analysis','thermal gravimetric analysis',
                      'XAFS','X-ray absorption fine structure',
                      'SAXS','Small-angle X-ray scattering',
                      'GC-MS','Gas chromatography–mass spectrometry',
                      'HPLC','High-performance liquid chromatography','high-pressure liquid chromatography',
                      'EXAFS','Extended X-Ray Absorption Fine Structure ',
                      'FTIR','Fourier-transform infrared spectroscopy',
                      'batch reaction testing',
                      'flow reaction testing',
                      'catalyst synthesis',
                      'SEM','Scanning Electron Microscopy',
                      'XAS','X-ray absorption spectroscopy',
                      'XPS','X-ray photoelectron spectroscopy',
                      'MD','Molecular dynamics',
                      'QM/MM','quantum mechanics/molecular mechanics',
                      'cluster model','Cluster analysis','clustering',
                      'FIB','Focused Ion Beam',
                      'XANES','X-ray absorption near edge structure', 
                      'NEXAFS','near edge X-ray absorption fine structure',
                      'DFT','density functional theory',
                      'XRD','X-ray Powder Diffraction',
                      'pre-edge','pre-edge of XANES spectrum',
                      'edge','edge of XANES spectrum',
                      'near-edge','near-edge of XANES spectrum',
                      'white line','white line of XANES spectrum',
                      'EXAFS','Extended X-Ray Absorption Fine Structure ',
                      'PDF','Pair Distribution Function']


## 3. PDFMiner search fucntions
The functions below use pdfminer and a list of keywords to look up for matches in publications

In [4]:
# functions for PDFminer

def get_pdf_text(pdf_file):
    return pdfmnr_hl.extract_text(pdf_file)

# get the paragraph fragments with references to data
def get_all_keyword_sentences(pdf_text, kwd_find):
    try:
        sentences = pdf_text.split("\n")
        groups=[]
        for sentence in sentences:
            if kwd_find.lower() in sentence.lower():
                idx = sentences.index(sentence)
                groups.append([idx-1, idx, idx+1])
        reduced_groups = []
        for group in groups:
            idx_group = groups.index(group)
            if groups.index(group) > 0:
                set_g = set(group)
                # make the array before current a set
                set_bg = set(groups[idx_group - 1])
                # make the array after current a set
                set_ag = set()
                if idx_group + 1 < len(groups):    
                    set_ag = set(groups[idx_group + 1])
                if len(set_bg.intersection(set_g)) > 0:
                    ordered_union = list(set_bg.union(set_g))
                    ordered_union.sort()
                    reduced_groups.append(ordered_union)
                if len(set_ag.intersection(set_g)) > 0:
                    ordered_union = list(set_ag.union(set_g))
                    ordered_union.sort()
                    reduced_groups.append(ordered_union)
                if len(reduced_groups) > 0:
                    is_in_rg = False
                    for a_rg in reduced_groups:
                        if set_g.issubset(a_rg):
                            is_in_rg = True
                            break
                    if not is_in_rg:
                        reduced_groups.append(list(set_g))
        return_group = []
        for sentence_group in reduced_groups:
            full_sentence = ""
            for single_sentence in sentence_group:
                full_sentence += sentences[single_sentence].strip()
            if not full_sentence in return_group:
                return_group.append(full_sentence)
    except:
        print("errot reading pdf")
    return return_group


## 4. Search in pdf files for terms

In [7]:
# provide the list of terms to search for

terms_to_find = list_of_techniques

net_zero = ['water', 'energy', 'waste minimisation', 'material reuse', 'gaseous emissions', 
            'circular economy', 'sustainable manufacturing', 'environment', 'innovation', 'water energy',
            'waste water', 'energy efficient', 'Energy and fuels from waste water', 
            'Life cycle sustainability', 'water shortages', 'changes in the climate',
            'sustainable energy supplies', 'life cycle analysis', 'treatment of water',
            'polluted water', 'sustainable energy', 'biomass', 'reducing the process energy demands', 
            'biofuels', 'Catalysis at the Water and Energy Nexus', 'chemical recycling', 'waste CO2', 
            'plastics recycle/reuse', 'conversion of carbon dioxide', 'renewable feedstocks', 
            'environmental impact', 'Earth-abundant metals', 'plastics will be recycled', 
            'high performance plastics', 'recycling of polymers', 'reusable monomers', 
            'sustainable polymer production', 'Catalysis for the Circular Economy and Sustainable Manufacturing',
            'more effective use of water and energy', 'reduction in gaseous emissions', 
            'End Of Life Plastic', 'ELP', 'Renewable Energy Technologies', 'Carbon Dioxide', 'Methanol', 
            'Hybrid Vehicle Emissions', 'emission control', 'pollutants', 'Solar-driven', 
            'regeneration', 'water treatment', 'microplastics', 'Reduction of Carbon Dioxide',
            'plastics waste', 'mixed plastics waste (MPW)', 'hydrogen', 'renewable hydrogen', 
            'Recycling of Waste Oxygenated Plastic', 'conversion of CO2',
            'catalyst modelling for the circular economy', 'solar fuels', 'emissions control',
            'water purification', 'particulate destruction in automotive exhaust', 
            'clean hydrogen production', 'energy materials', 'catalysis for deNOx reactions', 
            'hydrogen effect promotion of the CO oxidation', 'methane conversion', 
            'methanol conversion', 'renewable sources of energy', 'green hydrogen generation', 
            'renewables', 'sustainable alternatives', 'regenerative fuel cells', 
            'sustainable manufacturing of pharmaceuticals', 'sustainability considerations', 'waste plastics', 
            'recovery of energy', 'recycling process', 'marine debris', 
            'The Use of Sound for the Development of Water Electrolyser and Fuel', 'hydrogen production',
            'Hydrogen peroxide', 'clean chemical processes']
net_zero_singles = ['alternatives', 'automotive', 'automotive exhaust', 'biofuels', 'biomass', 'Carbon',
                    'carbon dioxide', 'Carbon Dioxide', 'Catalysis', 'cells', 'changes', 'chemical',
                    'circular', 'Circular', 'circular economy', 'clean', 'clean hydrogen', 'climate', 'CO',
                    'CO oxidation', 'CO2', 'considerations', 'control', 'conversion', 'debris', 'demands',
                    'deNOx', 'deNOx reactions', 'destruction', 'Development', 'Dioxide', 'Earth-abundant',
                    'economy', 'efficient', 'Electrolyser', 'ELP', 'emission', 'emissions', 'Emissions', 
                    'emissions', 'End Of Life', 'End Of Life Plastic', 'energy', 'environment', 
                    'environmental', 'exhaust', 'feedstocks', 'fuel', 'fuel cells', 'fuels', 'gaseous', 
                    'generation', 'green', 'high performance', 'Hybrid', 'hydrogen', 'Hydrogen peroxide', 
                    'innovation', 'Life cycle', 'manufacturing', 'Manufacturing', 'manufacturing', 'marine',
                    'material', 'methane', 'Methanol', 'methanol', 'microplastics', 'minimisation', 
                    'mixed plastic', 'mixed plastics', 'mixed plastics waste', 'monomers', 
                    'effective use', 'MPW', 'Nexus', 'oxidation', 'Oxygenated', 'Oxygenated Plastic', 
                    'particulate', 'particulate destruction', 'peroxide', 'pharmaceuticals', 'plastics', 
                    'pollutants', 'polluted', 'polymer', 'polymers', 'process', 'processes', 'production',
                    'promotion', 'purification', 'reactions', 'recovery', 'recycle', 'recycled', 'recycling',
                    'reducing', 'reduction', 'Reduction', 'regeneration', 'regenerative', 'renewable', 
                    'Renewable', 'renewable', 'renewables', 'reusable', 'reuse', 'shortages', 'solar', 
                    'solar fuels', 'Solar-driven', 'Sound', 'sources', 'supplies', 'sustainability', 
                    'sustainable', 'Sustainable', 'sustainable', 'treatment', 'Vehicle', 'Vehicle Emissions',
                    'waste', 'waste plastics', 'water']


terms_to_find = net_zero_singles
data_records = {}
term_mentions = {}
for a_pub in tqdm_notebook(db_pubs):
    data_refs = []
    data_sents = []
    pub_id = a_pub[0]
    pub_title = a_pub[1]
    pub_doi = a_pub[2]
    pub_url = a_pub[3]
    pub_pdf = a_pub[4]
    if pub_pdf == 'None':
        print("*************************")
        print("Missing PDF for:", pub_doi)
        print("*************************")
    else:
        pdf_file = "pdf_files/" + pub_pdf
        if not Path(pdf_file).is_file():
            print("*************************")
            print("Missing file for:", pdf_file, "for", pub_doi)
            print("*************************")
        else: 
            print("PDF filename", pdf_file)
            pdf_text = get_pdf_text(pdf_file)
            for a_term in terms_to_find:
                data_sentences = get_all_keyword_sentences(pdf_text,a_term)
                for d_sentence in data_sentences:
                    data_sents.append({'keyword':a_term,"desc":d_sentence, 'pub_id':pub_id})
    terms_in_pub = {}
    terms_in_pub['id'] = pub_id
    terms_in_pub['doi'] = pub_doi
    terms_in_pub['title'] = pub_title
    for sent in data_sents:
        if sent['keyword'] in terms_in_pub:
            terms_in_pub[sent['keyword']] +=1
        else:
            terms_in_pub[sent['keyword']] = 1
    
    term_mentions[pub_id]=terms_in_pub

<ipython-input-7-0875f06b8883>:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for a_pub in tqdm_notebook(db_pubs):


  0%|          | 0/494 [00:00<?, ?it/s]

PDF filename pdf_files/s41929-019-0334-3.pdf
errot reading pdf
PDF filename pdf_files/acscatal.9b00685.pdf
PDF filename pdf_files/cctc.201901268.pdf
PDF filename pdf_files/chem.201805250.pdf
PDF filename pdf_files/1-s2.0-S0968089618313233-main.pdf
PDF filename pdf_files/acssuschemeng.8b03568.pdf
PDF filename pdf_files/Smart anime donors.pdf
PDF filename pdf_files/C8OB00066B.pdf
errot reading pdf
PDF filename pdf_files/1-s2.0-S0968089617301268-main.pdf
PDF filename pdf_files/acs.biochem.8b00169.pdf
PDF filename pdf_files/acscatal.8b00389.pdf
PDF filename pdf_files/acscatal.8b00624.pdf
PDF filename pdf_files/jacs.7b12621.pdf
PDF filename pdf_files/acscatal.8b03169.pdf
PDF filename pdf_files/acscatal.9b01820.pdf
PDF filename pdf_files/C9CC02459J.pdf
errot reading pdf
PDF filename pdf_files/s41929-018-0213-3.pdf
errot reading pdf
PDF filename pdf_files/1-s2.0-S0926860X18305003-main.pdf
PDF filename pdf_files/1-s2.0-S0926860X18305817-main.pdf
PDF filename pdf_files/acs.jpcc.8b08420.pdf
PDF 

errot reading pdf
PDF filename pdf_files/C6TB01774F.pdf
errot reading pdf
PDF filename pdf_files/C5CC09780K.pdf
errot reading pdf
PDF filename pdf_files/C5TA09613H.pdf
errot reading pdf
PDF filename pdf_files/C5TA01592H.pdf
errot reading pdf
PDF filename pdf_files/C5CY02072G.pdf
errot reading pdf
PDF filename pdf_files/89fe74da010315dc464d8caf17bdc16a9a08.pdf
PDF filename pdf_files/C6CC01599A.pdf
errot reading pdf
PDF filename pdf_files/C5CC04188K.pdf
errot reading pdf
PDF filename pdf_files/C4GC00087K.pdf
errot reading pdf
PDF filename pdf_files/jp412501j.pdf
PDF filename pdf_files/10.1.1.1004.929.pdf
PDF filename pdf_files/C7DT02167D.pdf
errot reading pdf
PDF filename pdf_files/s41570-016-0002.pdf
PDF filename pdf_files/chem.201701013.pdf
PDF filename pdf_files/chem.201605690.pdf
PDF filename pdf_files/nature21001.pdf
PDF filename pdf_files/jacs.5b13070.pdf
PDF filename pdf_files/anie.201602930.pdf
PDF filename pdf_files/jacs.6b00710.pdf
PDF filename pdf_files/rsta.2015.0085.pdf
PDF 

errot reading pdf
PDF filename pdf_files/acsomega.9b03503.pdf
PDF filename pdf_files/cctc.201901166.pdf
PDF filename pdf_files/C9CY02371B.pdf
PDF filename pdf_files/d0sc01317j.pdf
errot reading pdf
*************************
Missing PDF for: 10.1016/b978-0-12-805324-9.09989-1
*************************
PDF filename pdf_files/1-s2.0-S0065237717300017-main.pdf
PDF filename pdf_files/s41563-019-0562-6.pdf
errot reading pdf
PDF filename pdf_files/D0CP90186E.pdf
*************************
Missing PDF for: 10.1142/q0035
*************************
PDF filename pdf_files/s41563-020-0800-y.pdf
errot reading pdf
PDF filename pdf_files/D0RA03871G.pdf
errot reading pdf
PDF filename pdf_files/D0CY00555J.pdf
errot reading pdf
PDF filename pdf_files/rsta.2020.0060.pdf
PDF filename pdf_files/d0sc02253e.pdf
errot reading pdf
PDF filename pdf_files/acscatal.0c00596.pdf
PDF filename pdf_files/D0SC02152K.pdf
errot reading pdf
PDF filename pdf_files/c9cp05476f.pdf
PDF filename pdf_files/acscatal.0c00414.pdf
PD

PDF filename pdf_files/ChemistryAEuropeanJ-2018-Kristinsdttir-BorylatedN-HeterocyclicCarbenesRearrangementAndChemicalTrapping.pdf
PDF filename pdf_files/10.1021_acs.jchemed.9b00181.pdf
PDF filename pdf_files/c9dt00535h.pdf
PDF filename pdf_files/ChemistryAEuropeanJ-2020-Caise-PartneringAThree-CoordinateGalliumCationWithAHydroborateCounter-IonForThe.pdf
PDF filename pdf_files/10.1002_ange.202011839.pdf
PDF filename pdf_files/10.1021_acsomega.0c00697.pdf
PDF filename pdf_files/d0dt01960g.pdf
errot reading pdf
PDF filename pdf_files/10.1002_celc.201902005.pdf
PDF filename pdf_files/ChemCatChem-2020-Valikhani-ImmobilisedEnzymesForSesquiterpeneSynthesisInBatchAndFlowSystems.pdf
PDF filename pdf_files/d0dt01447h.pdf
PDF filename pdf_files/d0cc02520h.pdf
errot reading pdf
PDF filename pdf_files/10.1002_cssc.201902340.pdf
PDF filename pdf_files/peerj-pchem-8.pdf
PDF filename pdf_files/catalysts-10-01067.pdf
PDF filename pdf_files/d1dt01216a.pdf
PDF filename pdf_files/d0ob02566f.pdf
errot readi

## 5. store terms found

In [8]:
if len(term_mentions) > 0:
        csvh.write_csv_data(term_mentions, 'net_zero_mentions202112d.csv')

## 6. search for terms using chem data extractor

{1: {'STEM': 4,
  'TEM ': 3,
  'XAFS': 10,
  'EXAFS': 6,
  'XPS': 8,
  'edge': 15,
  'doc_id': 1},
 2: {'XAS': 15, 'XPS': 14, 'XANES': 4, 'XRD': 24, 'edge': 5, 'doc_id': 2},
 3: {'SAXS': 21, 'doc_id': 3},
 4: {'MD': 6, 'doc_id': 4},
 5: {'HPLC': 6, 'SEM': 12, 'doc_id': 5},
 6: {'STEM': 4, 'SEM': 1, 'doc_id': 6},
 7: {'doc_id': 7},
 8: {'QM/MM': 1, 'DFT': 14, 'doc_id': 8},
 9: {'STEM': 2, 'doc_id': 9},
 10: {'DFT': 4, 'doc_id': 10},
 11: {'STEM': 7, 'doc_id': 11},
 12: {'doc_id': 12},
 13: {'STEM': 3, 'doc_id': 13},
 14: {'doc_id': 14},
 15: {'doc_id': 15},
 16: {'doc_id': 16},
 17: {'TEM ': 1,
  'XAFS': 27,
  'EXAFS': 30,
  'XANES': 23,
  'DFT': 8,
  'edge': 38,
  'doc_id': 17},
 18: {'doc_id': 18},
 19: {'XANES': 20, 'pre-edge': 36, 'edge': 49, 'doc_id': 19},
 20: {'doc_id': 20},
 21: {'TEM ': 2, 'XAFS': 14, 'EXAFS': 26, 'XRD': 4, 'edge': 7, 'doc_id': 21},
 22: {'doc_id': 22},
 23: {'STEM': 5,
  'TEM ': 5,
  'XAFS': 29,
  'EXAFS': 26,
  'XPS': 4,
  'XANES': 26,
  'XRD': 7,
  'edge': 1

## 5. store chem data extractor terms found

In [ ]:
import pdfminer
from pdfminer import high_level as pdfmnr_hl

# functions for PDFminer
def get_pdf_text(pdf_file):
    return pdfmnr_hl.extract_text(pdf_file)

# get the paragraph fragments with references to data
def get_ref_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if pr_fns.is_data_stmt(sentence.lower()):
            idx = sentences.index(sentence)
            groups.append([idx-1,idx,idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        return_group.append(full_sentence)
    return return_group

# get the paragraph fragments with references to data
def get_all_data_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if 'data' in sentence.lower() or 'inform' in sentence.lower():
            idx = sentences.index(sentence)
            groups.append([idx-1, idx, idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        if not full_sentence in return_group:
            return_group.append(full_sentence)
    return return_group

# get the http strings from references to data
def get_http_ref(sentence):
    http_frag = ""
    if 'http' in sentence.lower():
        idx_http = sentence.lower().index('http')
        http_frag = sentence[idx_http:]
        space_in_ref = True
        while " " in http_frag:
            space_idx = http_frag.rfind(" ")
            http_frag = http_frag[:space_idx]
        if(http_frag[-1:]=="."):
            http_frag = http_frag[:-1]
    return http_frag

In [ ]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
# library for getting data from crossref
import lib.crossref_api as cr_api
#library for handling json files
import json
# library for using regular expressions
import re
# library for handling http requests
import requests

# to iterate on lists
import itertools as itools

# validate urls
import validators

In [ ]:

# Custom Functions
# ** will migrate to lib if needed for more than one notebook

# get the crossreference json page from doi
def get_cr_json_object(cr_doi):
  crjd = None
  doi_file = 'json_files/' + cr_doi.replace('/','_').lower() + '.json'
  if not Path(doi_file).is_file():
    crjd = cr_api.getBibData(cr_doi)
    with open(doi_file, 'w', encoding='utf-8-sig', errors='ignore') as f:
                json.dump(crjd, f, ensure_ascii=False, indent=4)
  else:
    with open(doi_file, 'r', encoding='utf-8-sig') as jf:
        crjd = json.load(jf)
  # return the content and the file name 
  return crjd, doi_file

# get the landing page for the publication from uri
def get_pub_html_doi(cr_doi):
    html_file = 'html_files/' + cr_doi.replace('/','_').lower() + '.html'
    if not Path(html_file).is_file():
        page_content = urlh.getPageFromDOI(doi_text)
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(page_content.decode("utf-8") )
    else:
        f = open(html_file, "r")
        page_content = f.read()
    return page_content, html_file

# get a list of titles from the previous searches database
def get_titles(str_pub_title, db_name = "prev_search.sqlite3"):
    print(db_name)
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'prev_pop_searches'
    fields_required = "Num, Title"
    filter_str = "Title like '"+str_pub_title[0]+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

# get a list of ids, titles, and dois from the app database
def get_titles_and_dois(str_pub_title, db_name = "app_db.sqlite3"):
    print(db_name)
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, title, doi"
    filter_str = "Title like '"+str_pub_title[0]+"%';"
    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

# get a list of ids, titles, dois, links, pdf_file and 
# html_file names from the app database
def get_pub_app_data(db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, title, doi, link"
    filter_str = "status = 'Added'"
    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    #print(db_titles)
    search_in = 'pdf_files'
    filter_str = "id >= 1"
    fields_required = "id, doi, pdf_file"
    db_pdfs = db_conn.get_values(search_in, fields_required, filter_str)
    #print(db_pdfs)
    return_list = []
    zip_obj = itools.zip_longest(db_titles,db_pdfs)
    for an_art, a_pdf in zip_obj:
        if(a_pdf != None and an_art[0] == a_pdf[0]):
            return_list.append(tuple(an_art + (a_pdf[2], None)))
        else:
            return_list.append(tuple(an_art + (None, None)))
        
    db_conn.close()
    return return_list

# get the current csv working file
def get_working_file(nr_wf):
    working_file = wf_fields = None
    current_pass = 0
    if Path(nr_wf).is_file():
        working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
        for art_num in tqdm_notebook(working_file):
            if 'ignore' in working_file[art_num].keys():
                if current_pass < int(working_file[art_num]['ignore']):
                    current_pass = int(working_file[art_num]['ignore'])
            else:
                break
    print("Current pass:", current_pass)
    return working_file, wf_fields, current_pass

# get an htm file saved locally in the html_file folder 
def get_pub_html_url(text_url, entry_id):
    html_file = 'html_files/' +  entry_id + '.html'
    if not Path(html_file).is_file():
        print("")
        page_content = urlh.getPageFromURL(text_url)
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(page_content)
    else:
        f = open(html_file, "r")
        page_content = f.read()
    return page_content, html_file

# use regular expression to check if a given string
# is a valid DOI, using pattern from CR
def valid_doi(cr_doi):
    # CR DOIS: https://www.crossref.org/blog/dois-and-matching-regular-expressions/
    # CR DOIs re1
    # /^10.\d{4,9}/[-._;()/:A-Z0-9]+$/i
    if cr_doi == None:
        return False
    cr_re_01 = '^10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.match(cr_re_01, cr_doi, re.IGNORECASE)
    if compare != None and cr_doi == compare.group():
        return True
    else:
        return False
    
# get a semicolon separated list of authors from CR json data
def get_cr_author_list(article_data):
    authors = []
    if 'author' in article_data.keys():
        for author in article_data['author']:
            new_author=""
            new_author = author['family']
            if 'given' in author.keys():
                new_author += ", " + author['given']
            authors.append(new_author)
    return ("; ").join(authors)

# get the publication year from CR json data
def get_cr_year_published(article_data):
    year_print = 0
    if 'published-print' in article_data.keys() \
        and article_data['published-print'] != None \
        and article_data['published-print']['date-parts'][0] != None:
        year_print = int(article_data['published-print']['date-parts'][0][0])    
    elif 'journal-issue' in article_data.keys() \
        and article_data['journal-issue'] != None \
        and 'published-print' in article_data['journal-issue'].keys() \
        and article_data['journal-issue']['published-print'] != None \
        and article_data['journal-issue']['published-print']['date-parts'][0] != None:
        year_print = int(article_data['journal-issue']['published-print']['date-parts'][0][0])
    year_online = 0
    if 'published-online' in article_data.keys() \
        and article_data['published-online'] != None \
        and article_data['published-online']['date-parts'][0] != None:
        year_online = int(article_data['published-online']['date-parts'][0][0])    
    elif 'journal-issue' in article_data.keys() \
        and article_data['journal-issue'] != None \
        and 'published-online' in article_data['journal-issue'].keys() \
        and article_data['journal-issue']['published-online'] != None \
        and article_data['journal-issue']['published-online']['date-parts'][0] != None:
        year_print = int(article_data['journal-issue']['published-online']['date-parts'][0][0])
    if year_print != 0 and year_online != 0:
        return year_print if year_print < year_online else year_online
    else:
        return year_print if year_online == 0 else year_online
    return 0

# try to download a pdf from a given url
def get_pdf_from_url(pdf_url):
    fname = ""
    try:
        response = requests.get(pdf_url)
        content_type = response.headers['content-type']
        print("Content Type:", content_type )
        print(response.headers)
        cd= response.headers['content-disposition']
        print(cd)
        fname = re.findall("filename=(.+)", cd)[0]
        print(fname)
        if not 'text' in content_type:
            with open('pdf_files/'+ fname +'.pdf', 'wb') as f:
                f.write(response.content)
    except:
        print("Error getting file from: ", pdf_url)
    finally:
        return fname
# add name of the pdf file for a publication record in the app database     
def set_pdf_file_value(file_name, pub_id, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    done = db_conn.set_value_table(table, pub_id, "pdf_file", file_name)
    db_conn.close()
    return done

# try to get a pdf from elsevier
def get_elsevier_pdf(doi):
    pdf_url = f'http://api.elsevier.com/content/article/doi:{doi}?view=FULL'
    print("\t", pdf_url) 
    return get_pdf_from_url(pdf_url)

# try to get a pdf from wiley
def get_wiley_pdf(doi):
    pdf_url = f'https://onlinelibrary.wiley.com/doi/pdf/{doi}'
    print("\t", pdf_url) 
    return get_pdf_from_url(pdf_url)

def get_not_matched_files(db_name):
    files_list = get_files_list(Path("pdf_files"))
    db_pubs = get_pub_app_data(db_name)
    missing=[]
    # check which files are really missing linking
    for file in files_list:
        found_in_db = False
        for db_pub in db_pubs:
            if file.name == db_pub[4]:
                found_in_db = True
                break
        if not found_in_db:
           missing.append(file) 
    return missing

In [ ]:
# functions for ChemDataExtractor
# not used for mining data references (suplementary/raw) or to get pdf metadata
from chemdataextractor import Document

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

def cde_read_pdfs(a_file):
    pdf_f = open(a_file, 'rb')
    doc = Document.from_file(pdf_f)
    return doc

def find_doi(element_text):
    cr_re_01 = '10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.search(cr_re_01, element_text, re.IGNORECASE)
    if compare != None:
        return compare.group()
    return ""

def get_db_id(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "id", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_db_title(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "title", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_close_dois(str_name, db_name = "prev_search.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, doi, title, pdf_file"
    filter_str = "doi like '%"+str_name+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

Get the name of the current app db file:

In [ ]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = "db_files/app_db20211005.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()

## Get pdf files for publications

Read database and try to recover pdf files

In [ ]:
# get publication data from the ukch app
db_pubs = get_pub_app_data(ukchapp_db)
for a_pub in tqdm_notebook(db_pubs):
    if a_pub[0] > 688:
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        pub_pdf = a_pub[4]
        pub_html = a_pub[5]
        if pub_pdf == None:
            not_in_url = True
            print("ID: ", pub_id, "Publication: ",pub_title,
                  "\n\tDOI: ", pub_doi, " URL: ", pub_url)
            if "pdf" in pub_url and validators.url(pub_url):
                print ("\tTry to get the pdf from URL: ", pub_url)
                try:
                    response = requests.get(pub_url)
                    content_type = response.headers['content-type']
                    if not 'text' in content_type:
                        #print(response.headers)
                        cd= response.headers['content-disposition']
                        #print(cd)
                        fname = re.findall("filename=(.+)", cd)[0]
                        #print(fname)
                        if not Path('pdf_files/' + pdf_file).is_file():
                            with open('pdf_files/'+ fname +'.pdf', 'wb') as f:
                                f.write(response.content)
                        else:
                            set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                        not_in_url = False
                except:
                    print("ID: ", pub_id, "\nPublication: ",pub_title, 
                           "\nDOI: ", pub_doi, "\nURL: ", pub_url) 
            if not_in_url:
                print("\tTry to see if json file has link to pdf: ")
                if valid_doi(pub_doi):
                    crjd, doi_file = get_cr_json_object(pub_doi)
                    got_pdf = False
                    if "link" in crjd.keys():
                        for a_link in crjd["link"]:
                            if "\tURL" in a_link.keys() and ("pdf" in a_link["URL"] or "pdf" in a_link["content-type"]):
                                cr_url = a_link["URL"]
                                #print("URL: ", cr_url)
                                pdf_file = get_pdf_from_url(cr_url)
                                # if the name corresponds to a existing file, assign value to db_record
                                if Path('pdf_files/' + pdf_file).is_file():
                                    print("\tFile name:", pdf_file)
                                    set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                                    got_pdf = True
                                else:
                                    print("\tcould not get file from", cr_url)
                    else: 
                        print("\tno links in json", pub_doi)
                if not got_pdf and "elsevier" in pub_url:
                    print("\tTrying elsevier doi:" )
                    pdf_file = get_elsevier_pdf(pub_doi)
                    if Path('pdf_files/' + pdf_file).is_file():
                        print("\tFile name:", pdf_file)
                        set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                        got_pdf = True
                elif not got_pdf and "wiley" in pub_url:
                    print("\tTrying wiley doi:" )
                    pdf_file = get_wiley_pdf(pub_doi)
                    if Path('pdf_files/' + pdf_file).is_file():
                        print("\tFile name:", pdf_file)
                        set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                        got_pdf = True

                if not got_pdf:
                    print("\tTry doi:  https://doi.org/" + pub_doi)


In [ ]:
import urllib.request
opener = urllib.request.build_opener()
opener.addheaders = [('Accept', 'application/vnd.crossref.unixsd+xml')]
r = opener.open('http://dx.doi.org/10.1002/anie.201901592')
print (r.info()['Link'])

import json
s=[{"URL"=>"https://api.wiley.com/onlinelibrary/tdm/v1/articles/10.1002%2Fanie.201901592", "content-type"=>"application/pdf", "content-version"=>"vor", "intended-application"=>"text-mining"}, {"URL"=>"https://onlinelibrary.wiley.com/doi/pdf/10.1002/anie.201901592", "content-type"=>"application/pdf", "content-version"=>"vor", "intended-application"=>"text-mining"}, {"URL"=>"https://onlinelibrary.wiley.com/doi/full-xml/10.1002/anie.201901592", "content-type"=>"application/xml", "content-version"=>"vor", "intended-application"=>"text-mining"}, {"URL"=>"https://onlinelibrary.wiley.com/doi/pdf/10.1002/anie.201901592", "content-type"=>"unspecified", "content-version"=>"vor", "intended-application"=>"similarity-checking"}]
d = json.loads(s)

print(d)

## File name match

In [ ]:
# check if file name matches some part of a doi
files_list = get_not_matched_files(ukchapp_db)

not_assigned = []
for a_file in tqdm_notebook(files_list):
    search_this = a_file.name.replace(".pdf", "").lower()
    print(a_file.name,"\t",search_this)
    close_dois = get_close_dois(search_this, ukchapp_db)
    print(len(close_dois))
    
    if len(close_dois) == 1 :
        doi_dat = close_dois[0]
        selected = False
        if doi_dat[3] == None:
            while not selected:
                print("Assign file: ", a_file.name, " to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                print('***************************************************************')
                print("Options:\n\ta) assign\n\tb)go to next")
                print("selection:")
                usr_select = input()
                if usr_select == 'a':
                    selected = True
                    set_pdf_file_value(a_file.name, doi_dat[0], ukchapp_db)
                    print("assing and go to next")
                elif usr_select == 'b':
                    #working_file[art_num]['ignore']=3 # visual inspection
                    selected = True
                    print("going to next")
        else:
            print("Assigned in db: ",  doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
    else:
        not_assigned.append(a_file)        

## Use pdfminer to get metadata from pdf file

In [ ]:
files_list = get_files_list(Path("pdf_files"))
db_pubs = get_pub_app_data(ukchapp_db)
missing=[]
# check which files are really missing linking
for file in files_list:
    found_in_db = False
    for db_pub in db_pubs:
        if file.name == db_pub[4]:
            found_in_db = True
            break
    if not found_in_db:
        missing.append(file)

# check if all linked files are in the folder
missing2=[]
for db_pub in db_pubs:
    found_in_system = False
    for file in files_list:
        if file.name == db_pub[4] or db_pub[4] == None:
            found_in_system = True
            break
    if not found_in_system:
        missing2.append(db_pub)


In [ ]:
# use ChemDataExtractor to read pdf and get DOIs in document
for a_file in tqdm_notebook(not_assigned):
    pdf_doc = cde_read_pdfs(a_file)
    print(a_file.name)
    dois_list = []
    for element in pdf_doc.elements:
        if 'doi' in str(element):
            found_doi = find_doi(str(element))
            if found_doi[-1:] == ".":
                found_doi = found_doi[:-1]
            if not found_doi in dois_list:
                dois_list.append(found_doi)       
    
    if dois_list != [] and len(dois_list) == 1:
        for a_doi in dois_list:
            close_dois = get_close_dois(a_doi, ukchapp_db)
            selected = False
            if len(close_dois) == 1:
                doi_dat = close_dois[0]
                if doi_dat[3] == None:
                    while not selected:
                        print("Assign file: ",a_file.name, " to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                        print('***************************************************************')
                        print("Options:\n\ta) assign\n\tb)go to next")
                        print("selection:")
                        usr_select = input()
                        if usr_select == 'a':
                            selected = True
                            set_pdf_file_value(a_file.name, doi_dat[0], ukchapp_db)
                            print("assing and go to next")
                        elif usr_select == 'b':
                            #working_file[art_num]['ignore']=3 # visual inspection
                            selected = True
                            print("going to next")
                else: 
                    print("Already assingned to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                

In [ ]:
# patch to update most recent version of app DB
ukchapp_db = "db_files/app_db2.sqlite3"
ukchapp_db_prev = "db_files/app_db.sqlite3"
db_pubs = get_pub_app_data(ukchapp_db)
db_pubs_prev = get_pub_app_data(ukchapp_db_prev)

for a_pub in tqdm_notebook(db_pubs):
    for old_pub in db_pubs_prev:
        if old_pub[0] == a_pub[0]:
            if a_pub[4] != None:
                print ("************** Assigned:", a_pub[4], old_pub[4])
            elif a_pub[4] == None:
                print("Assign file: ", old_pub[4], " to:\n\t",  a_pub[0], a_pub[1], a_pub[2]  )
                set_pdf_file_value(old_pub[4], a_pub[0], ukchapp_db)
                
            
        